<a href="https://colab.research.google.com/github/talhaanwarch/Offensive-Language-Detection/blob/master/OLID_ELMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/dataset/OLID

/content/drive/My Drive/dataset/OLID


In [3]:
ls

elmo-model.h5      labels-levelb.csv       testset-levelb.tsv
glove.6B.100d.txt  labels-levelc.csv       testset-levelc.tsv
glove.6B.200d.txt  olid-training-v1.0.tsv
labels-levela.csv  testset-levela.tsv


In [4]:
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import string
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
train=pd.read_csv( 'olid-training-v1.0.tsv',sep="\t")
test=pd.read_csv('testset-levela.tsv',sep="\t")
y_test=pd.read_csv( 'labels-levela.csv',header=None).iloc[:,-1]
#OFF=0 
#NOT=1

In [6]:
y_train=train['subtask_a']
train=train['tweet']
test=test['tweet']
Y_train=pd.factorize(y_train)[0]
Y_test=pd.factorize(y_test)[0]

import collections
collections.Counter(y_train)

#Counter({0: 4400, 1: 8840})

Counter({'NOT': 8840, 'OFF': 4400})

In [0]:
# Creating the training corpus
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()
corpus_train = []
for i in train:
    x=i.lower()
    x=x.replace('@user','')
    x=x.replace('@[\w\-]+','')
    #x=x.translate(str.maketrans('', '', string.punctuation))
    x = re.sub('[^A-Za-z]', ' ', x)
    #x=re.sub('\s+',' ',x)
    x=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',x) #url
    x = [lemmatizer.lemmatize(token) for token in x.split(" ")]
    x = [word for word in x if not word in stop_words]
    x=" ".join(x)
    corpus_train.append(x)    
# Creating the training corpus
corpus_test = []
for i in test:
    x=i.lower()
    x=x.replace('@user','')
    x=x.replace('@[\w\-]+','')
    #x=x.translate(str.maketrans('', '', string.punctuation))
    x = re.sub('[^A-Za-z]', ' ', x)
    #x=re.sub('\s+',' ',x)
    x=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',x) #url
    x = [lemmatizer.lemmatize(token) for token in x.split(" ")]
    x = [word for word in x if not word in stop_words]
    x=" ".join(x)
    corpus_test.append(x) 

In [8]:
X_train=pd.DataFrame(corpus_train) 
X_train.columns=['tweet']
X_train.head()

,tweet
0,ask native american take
1,go home drunk maga trump url
2,amazon investigating chinese employee selling ...
3,someone vetaken piece shit volcano
4,obama wanted liberal amp illegals move red...


In [9]:
freq = pd.Series(' '.join(X_train['tweet']).split()).value_counts()[-10:]
X_train['tweet']= X_train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
X_train['tweet'].head()

0                             ask native american take
1                         go home drunk maga trump url
2    amazon investigating chinese employee selling ...
3                   someone vetaken piece shit volcano
4     obama wanted liberal amp illegals move red state
Name: tweet, dtype: object

In [0]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [11]:
import tensorflow_hub as hub
import tensorflow as tf

embed = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [12]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K
from sklearn.utils import class_weight
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense1 = Dense(500, activation='relu')(embedding)
dense2 = Dense(250, activation='relu')(dense1)
pred = Dense(1, activation='sigmoid')(dense2)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))
model.fit(X_train,Y_train,batch_size=128,epochs=10,verbose=2,class_weight=class_weights,validation_split=0.2)





Using TensorFlow backend.


Train on 10592 samples, validate on 2648 samples
Epoch 1/10
 - 35s - loss: 0.5796 - acc: 0.6885 - val_loss: 0.5163 - val_acc: 0.6979
Epoch 2/10
 - 28s - loss: 0.5128 - acc: 0.7200 - val_loss: 0.5046 - val_acc: 0.7198
Epoch 3/10
 - 29s - loss: 0.4872 - acc: 0.7426 - val_loss: 0.5340 - val_acc: 0.7126
Epoch 4/10
 - 29s - loss: 0.4812 - acc: 0.7477 - val_loss: 0.4910 - val_acc: 0.7428
Epoch 5/10
 - 29s - loss: 0.4631 - acc: 0.7594 - val_loss: 0.4995 - val_acc: 0.7394
Epoch 6/10
 - 29s - loss: 0.4476 - acc: 0.7710 - val_loss: 0.5381 - val_acc: 0.7508
Epoch 7/10
 - 28s - loss: 0.4475 - acc: 0.7758 - val_loss: 0.5080 - val_acc: 0.7572
Epoch 8/10
 - 29s - loss: 0.4266 - acc: 0.7901 - val_loss: 0.5015 - val_acc: 0.7485
Epoch 9/10
 - 28s - loss: 0.3965 - acc: 0.8050 - val_loss: 0.5468 - val_acc: 0.7474
Epoch 10/10
 - 29s - loss: 0.3781 - acc: 0.8188 - val_loss: 0.5346 - val_acc: 0.7545


In [13]:
X_test=pd.DataFrame(corpus_test) 
X_test.columns=['tweet']
X_train.head()
freq = pd.Series(' '.join(X_test['tweet']).split()).value_counts()[-10:]
X_test['tweet']= X_train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
X_test['tweet'].head()

0                                                                                                                              ask native american take
1                                                                                                                          go home drunk maga trump url
2    amazon investigating chinese employee selling internal data third party seller looking edge competitive marketplace url amazon maga kag china tcot
3                                                                                                                    someone vetaken piece shit volcano
4                                                                                                      obama wanted liberal amp illegals move red state
Name: tweet, dtype: object

In [14]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred = np.round(y_pred)
#y_pred = (y_pred > 0.5)

print(classification_report(Y_test, y_pred))


860/860 [==============================] - 3s 3ms/step
              precision    recall  f1-score   support

           0       0.26      0.21      0.23       240
           1       0.72      0.77      0.74       620

    accuracy                           0.62       860
   macro avg       0.49      0.49      0.49       860
weighted avg       0.59      0.62      0.60       860



In [0]:
import keras
keras.backend.clear_session()